# Retrive box office revenue and profit data

Shenyue Jia

**Task lists**

- [ ] Obtain the following data from tmdb
    - Revenue
    - Budget
    - Certification (P, PG, etc.)
- [ ] Perform Explanatory Data Analysis

In [1]:
# data wrangling
import numpy as np
import pandas as pd
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

## Get movie info of a pre-defined subset

- We will use movie retrieved previously from [IMDB](https://www.imdb.com)
- Data can be found from the repository of this project ([link](https://github.com/jiashenyue/project3-imdb-data/blob/main/Data/title_ratings.csv.gz))

In [2]:
# read the rating file to obtain the tconst of each movie for analysis
df_basics = pd.read_csv('Data/title_basics_filtered.csv.gz')
df_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [3]:
# how many movies we will analysis
print(f'There are {df_basics.shape[0]} movies in the df_rating dataframe.')

There are 137367 movies in the df_rating dataframe.


## Extract data from TMDB

- [TMDB](https://www.themoviedb.org/) is a database with more information of movies based on the skeleton information available from [IMDB](https://www.imdb.com/). Information available from TMDB includes
    - Box office revenue generated by a movie
    - Profit data of a movie
- In this section, we will use data from TMDB to further analyze the pattern of best-selling movies

### Set up TMDB API and relevant python packages

- We will use `tmdbsimple` package to make our life easier to extract data from TMDB API without manually constructing the URLs for our API calls.
    - [GitHub repository of `tmdbsimple`](https://github.com/celiao/tmdbsimple)
    - [PyPi link of `tmdbsimple`](https://pypi.org/project/tmdbsimple/)

In [4]:
# Install tmdbsimple (only need to run once)
# !pip install tmdbsimple

In [5]:
with open('/Users/Shenyue/.secret/tmdb.api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['API Key'])

In [6]:
tmdb.API_KEY =  login['API Key']

### Make some test queries to understand the TMDB data

In [7]:
## make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

In [8]:
## movie objects have a .info dictionary 
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/waCRuAW5ocONRehP556vPexVXA9.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 72.162,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

In [9]:
# print revenue
info['revenue']

463517383

In [10]:
# print budget
info['budget']

63000000

In [11]:
# print imdb id
info['imdb_id']

'tt0133093'

- The `imdb_id` field in tmdb data is the `tconst` field in the data we retrieved from IMDB
- We will see how to obtain the certification info from data

In [12]:
# example from package README
# source = https://github.com/celiao/tmdbsimple
releases = movie.releases()
for c in releases['countries']:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])

R
R


- Instead of printing the certification info, we will add the certification info to the `info` dictionary

In [13]:
# Get the movie object for the current id
movie = tmdb.Movies('tt1361336')
# save the .info .releases dictionaries
info = movie.info()
releases = movie.releases()
# Loop through countries in releases
for c in releases['countries']:
    # if the country abbreviation==US
    if c['iso_3166_1' ] =='US':
        ## save a "certification" key in the info dict with the certification
       info['certification'] = c['certification']

### Prepare a function to obtain movie info

- Wrap the above tests into a function to return an info dictionary of a given `movie_id`

In [14]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # Save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            ## Save a "certification" key in info with the certification
            info['certification'] = c['certification']
    
    return info

In [15]:
# test the above function to see if certification is added to info
test = get_movie_with_rating("tt0848228") #put your function name here
test

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 185.145,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [21]:
## test our function by looping through a list of ids
imdb_ids = ["tt0848228","tt0332280"]
results = []
errors = []
for movie_id in imdb_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except Exception as e: 
        errors.append([movie_id, e])
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28260,PG-13
1,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28260,PG-13
2,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.878,10184,PG-13


## Obtain required data for all movies in `df_basics`

- `df_basics` contains movies that are already filtered using the following conditions from the original IMDB database
    - Exclude any movie with missing values for genre or runtime
    - Include only full-length movies
    - Include only fictional movies (not from documentary genre)
    - Include only movies that were released 2000 - 2021
    - Include only movies that were released in the United States
- We will use the function above to obtain the following information for all movies in `df_basics`
    - `revenue`
    - `budget`
    - `certification`

In [22]:
basics = pd.read_csv('Data/title_basics_filtered.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


### Write a test query with year `2000` or `2001`

- Write a function to write JSON file obtained from API calls

In [28]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.DS_Store',
 'title_basics_filtered.csv.gz',
 'title_basics_large.csv.gz',
 'title_ratings_large.csv.gz',
 'title_akas_filtered.csv.gz',
 'title_ratings_filtered.csv.gz',
 'title_akas_large.csv.gz']

In [26]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [23]:
YEARS_TO_GET = [2000,2001]

In [24]:
# define an error list to store error messages
erros = []

In [32]:
# Start of OUTER loop
# loop over each year to get
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    
    # Specifying JSON_FILE filename (can include a folder)
    JSON_FILE = FOLDER+f"tmdb_api_results_{YEAR}.json"
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    ## if no, Create the empty JSON file with with open that just contains 
    ## the key "imdb_id"
    if file_exists == False:
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok = True)
        
        ## save the first page of results
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    
    ## if yes, do nothing
    ## If it exists, inform user
    else:
        print(f"[i] {JSON_FILE} already exists.")
    
    ## define IDs to call
    #Saving new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
   
    ## filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    ## Start of INNER loop
    # Get index and movie id from list
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
    
    ## save the current JSON file to a compressed format
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", 
                         compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/2737 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/2859 [00:00<?, ?it/s]

In [33]:
# Print a message reporting back the number of movies caused an error
print(f"- Total errors: {len(errors)}")

- Total errors: 1109


## Reference

- Codes are adapted from Coding Dojo Data Enrichment tutorials for Project 3
    - [Intro to TMDB API](https://login.codingdojo.com/m/376/12529/88081)
    - [Efficient TMDB API Calls](https://login.codingdojo.com/m/376/12529/88082)
    - [Project 3-Part 2: Extract from TMDB](https://login.codingdojo.com/m/376/12529/88084)